## cyber~rank

In [1]:
import math
import collections
import functools
import operator

Lets say 4 agents in the network with resource token balances (2.000, 7.000, 9.000, 5.000) accor

In [2]:
agents = {
    'agent_1': 2.000,
    'agent_2': 7.000,
    'agent_3': 9.000,
    'agent_4': 5.000,
}

Named agents create cyberlinks:

In [3]:
agents_cyberlinks = {
    'agent_1': [(1, 2), (3, 4)],
    'agent_2': [(2, 3)],
    'agent_3': [(1, 3), (2, 4), (3, 4)],
    'agent_4': [(4, 2), (2, 1)],
}

Let's define a function to prepare the context for cyber~rank and entropy calculations:

In [4]:
def context_preparator(agents: dict, agents_cyberlinks: dict) -> list:
    cyberlinks_wihth_agent_weight = []
    for agent, cyberlinks in agents_cyberlinks.items():
        temp = (cyberlinks, agents[agent])
        cyberlinks_wihth_agent_weight.append(temp)
    grouped_cyberlinks = collections.Counter()
    for agent_cyberlinks in cyberlinks_wihth_agent_weight:
        cyberlinks = agent_cyberlinks[0]
        agent_weight = agent_cyberlinks[1]
        agent_cyberlink_weight = agent_weight / len(cyberlinks)
        agent_weighted_cyberlinks = {cyberlink: agent_cyberlink_weight for cyberlink in cyberlinks}
        grouped_cyberlinks += collections.Counter(agent_weighted_cyberlinks)
    cyberlinks = [{k:v} for k,v in grouped_cyberlinks.items()]
    return cyberlinks

And now let prepare context for the rank and entropy calculations

In [6]:
cyberlinks = context_preparator(agents, agents_cyberlinks)
cyberlinks

[{(1, 2): 1.0},
 {(3, 4): 4.0},
 {(2, 3): 7.0},
 {(1, 3): 3.0},
 {(2, 4): 3.0},
 {(4, 2): 2.5},
 {(2, 1): 2.5}]

Define cyber rank function:

In [7]:
def rank_calc(cyberlinks: list, tolerance: float = 0.001, damping_factor: float = 0.8):
    cyberlinks_dict = dict(functools.reduce(operator.add, map(collections.Counter, cyberlinks)))
    objects = list(set([item for t in [list(x.keys())[0] for x in cyberlinks] for item in t]))
    rank = [0] * len(objects)
    size = len(objects)
    default_rank = (1.0 - damping_factor) / size
    dangling_nodes = [obj for obj in objects if obj not in [list(cyberlink.keys())[0][1] for cyberlink in cyberlinks]]
    dangling_nodes_size = len(dangling_nodes)
    inner_product_over_size = default_rank * (dangling_nodes_size / size)
    default_rank_with_correction = (damping_factor * inner_product_over_size) + default_rank
    change = tolerance + 1

    steps = 0
    prevrank = [0] * len(objects)
    while change > tolerance:
        for obj in objects:
            obj_index = objects.index(obj)
            ksum = 0
            income_cyberlinks = [income_cyberlink for income_cyberlink in [list(x.keys())[0] for x in cyberlinks] if income_cyberlink[1] == obj]
            for cyberlink in income_cyberlinks:
                linkStake = cyberlinks_dict[cyberlink]
                outcome_cyberlinks = [outcome_cyberlink for outcome_cyberlink in [list(x.keys())[0] for x in cyberlinks] if outcome_cyberlink[0] == cyberlink[0]]
                jCidOutStake = sum([cyberlinks_dict[outcome_cyberlink] for outcome_cyberlink in outcome_cyberlinks])
                if linkStake == 0 or jCidOutStake == 0:
                    continue
                weight = linkStake / jCidOutStake
                ksum = prevrank[obj_index] * weight + ksum
            rank[obj_index] = ksum * damping_factor + default_rank_with_correction
        change = abs(max(rank) - max(prevrank))
        prevrank = rank
        steps += 1
    res = list(zip(objects, rank))
    res.sort(reverse=True, key=lambda x: x[1])
    return res

In [8]:
rank_calc(cyberlinks)

[(3, 0.10239999999999999),
 (2, 0.09999999999999998),
 (4, 0.09959999999999998),
 (1, 0.05799999999999999)]

# Entropy calculation

In [9]:
def entropy_calc(cyberlinks: list, damping_factor: float = 0.8):
    cyberlinks_dict = dict(functools.reduce(operator.add, map(collections.Counter, cyberlinks)))
    objects = list(set([item for t in [list(x.keys())[0] for x in cyberlinks] for item in t]))
    swd = [0] * len(objects)
    sumswd = [0] * len(objects)
    entropy = [0] * len(objects)
    for i in range(len(swd)):
        income_cyberlinks = [income_cyberlink for income_cyberlink in [list(x.keys())[0] for x in cyberlinks] if income_cyberlink[1] == objects[i]]
        outcome_cyberlinks = [outcome_cyberlinks for outcome_cyberlinks in [list(x.keys())[0] for x in cyberlinks] if outcome_cyberlinks[0] == objects[i]]
        swd[i] = damping_factor * sum(cyberlinks_dict[income_cyberlink] for income_cyberlink in income_cyberlinks) + \
                 (1 - damping_factor) * sum(cyberlinks_dict[outcome_cyberlink] for outcome_cyberlink in outcome_cyberlinks)
    for i in range(len(sumswd)):
        income_cyberlinks = [income_cyberlink for income_cyberlink in [list(x.keys())[0] for x in cyberlinks] if income_cyberlink[1] == objects[i]]
        for income_cyberlink in income_cyberlinks:
            obj = objects.index(income_cyberlink[0])
            sumswd[i] += damping_factor * swd[obj]
        outcome_cyberlinks = [outcome_cyberlinks for outcome_cyberlinks in [list(x.keys())[0] for x in cyberlinks] if outcome_cyberlinks[0] == objects[i]]
        for outcome_cyberlink in outcome_cyberlinks:
            obj = objects.index(outcome_cyberlink[1])
            sumswd[i] += (1 - damping_factor) * swd[obj]
    for i in range(len(entropy)):
        if swd[i] == 0:
            continue
        income_cyberlinks = [income_cyberlink for income_cyberlink in [list(x.keys())[0] for x in cyberlinks] if income_cyberlink[1] == objects[i]]
        for income_cyberlink in income_cyberlinks:
            obj = objects.index(income_cyberlink[0])
            if sumswd[obj] == 0:
                continue
            entropy[i] += abs(-swd[i] / sumswd[obj] * math.log2(swd[i]/sumswd[obj]))
        outcome_cyberlinks = [outcome_cyberlinks for outcome_cyberlinks in [list(x.keys())[0] for x in cyberlinks] if outcome_cyberlinks[0] == objects[i]]
        for outcome_cyberlink in outcome_cyberlinks:
            obj = objects.index(outcome_cyberlink[1])
            if sumswd[obj] == 0:
                continue
            entropy[i] += abs(-swd[i] / sumswd[obj] * math.log2(swd[i] / sumswd[obj]))
    res = list(zip(objects, entropy))
    res.sort(reverse=True, key=lambda x: x[1])
    return res

In [68]:
# cyberlinks = [
#     {(1, 2): 8},
#     {(1, 15): 6},
#     {(3, 2): 16},
#     {(4, 3): 8},
#     {(5, 4): 4},
#     {(5, 6): 6},
#     {(7, 6): 6},
#     {(8, 7): 4},
#     {(9, 8): 4},
#     {(9, 10): 4},
#     {(10, 11): 4},
#     {(11, 12): 6},
#     {(12, 19): 9},
#     {(13, 12): 6},
#     {(13, 14): 8},
#     {(15, 14): 12},
#     {(15, 2): 12},
#     {(16, 2): 8},
#     {(16, 14): 8},
#     {(17, 3): 8},
#     {(17, 14): 8},
#     {(18, 3): 8},
#     {(18, 19): 6},
#     {(20, 19): 6},
#     {(21, 20): 4},
#     {(21, 6): 6}
# ]

In [10]:
entropy_calc(cyberlinks)

[(2, 2.0393614893359167),
 (1, 1.5439077306570839),
 (4, 1.1878887843191328),
 (3, 0.9723731309047978)]

## Resource tokens minting

In [61]:
def mint_resource_token(
                        hydrogen: int, 
                        investmint_time: int, 
                        base_vesting_resource: int = 10_000_000_000, 
                        base_vesting_time: int = 86400,
                        max_vesting_time: int = 604800):
    # hydrogen is a fuel token for resources minting
    # investment time is a time token locked
    if investmint_time > max_vesting_time:
        print('Impossible to lock tokens longer than max vesting time')
        return 0
    return int(hydrogen / base_vesting_resource) * (investmint_time / base_vesting_time)

In [62]:
mint_resource_token(100_000_000_000, 86400)

10.0